CDMA　シミュレーション
===

In [84]:
# import
import numpy as np
import matplotlib.pyplot as plt
import sys

np.set_printoptions(precision=4, suppress=True, floatmode="maxprec_equal")
plt.rcParams['figure.figsize'] = (12,4)

In [85]:
# パラメータ
Ndata = 1000
Nchip = 100
UserPower = np.array([1, 100])
Nuser = max(UserPower.shape)
SNRdB = 20
Lss = Ndata * Nchip;

In [86]:
def rndCode(codeSize, Type):
    """
    codeSize: 生成するランダム系列の大きさ（行数ｘ列数）を指定
    Type: タイプ指定 0: [0,1], 1: [-1, 1]
    """
    
    rndCode = np.random.randn(*codeSize)
    
    if rndCode.ndim > 2:
        sys.exit()
    
    dataSize = rndCode.shape
    if len(dataSize) == 1:
        sys.exit()
    
    if Type == 0:
        rndCode = np.where(rndCode <= 0, 0, 1)
    else:
        rndCode = np.where(rndCode <= 0, -1, 1)
    
    return rndCode

# BPSK変調器
def bpskMod(data):
    """
    data: 列ベクトル
    bpskSymbol： 列ベクトル
    """
    bpskSymbol = data.copy()
    np.place(bpskSymbol, bpskSymbol == 0, -1)
    
    return bpskSymbol

# ガウス雑音生成

def awgn(Pn, rn, cn):
    """
    Pn: 雑音電力
    ｒｎ: 行数
    cn: 列数
    """
    n = np.random.randn(rn, cn) + 1j*np.random.randn(rn, cn)
    n = n * np.sqrt(Pn/2)
    
    return n

# 相関器
def correlator(w, rSig):
    """
    w: 重み係数
    sig:　入力信号
    """
    sig = rSig.copy()
    
    sizeW = w.shape
    Lw = sizeW[0] # 100
    Nw = sizeW[1] # 1
    corIn = np.vstack([sig, np.zeros((Lw,1))]) # 1100 x 1
    corOut = np.zeros((max(sig.shape), Nw), dtype="complex") # 1000x1
    
    for co in range(0, max(corOut.shape)):
        corOut[co, :] = (w.conj().T @ corIn[co:(co + Lw)])[0]

    return corOut

# BPSK復調器
def bpskDem(rSig):
    """
    rSig : 受信信号
    rData:　受信データ
    """
    rData = np.ones(rSig.shape)
    rData[rSig < 0] =  0
    
    return rData

# BER比較器
def ber(data1, data2):
    BER = np.sum(np.abs(data1-data2))/max(data1.shape)
    return BER

In [87]:
# 送信機
data = rndCode([Ndata, Nuser], 0)
bpskSymbol = bpskMod(data)
Nsymbol = max(bpskSymbol.shape)
spCode = rndCode([Nchip, Nuser], 1)
ssSig = spCode @ np.diag(np.sqrt(UserPower)) @ bpskSymbol.T
ssSig = np.array([ssSig.flatten('F')]).T

In [88]:
# 通信経路
Pn = 10 ** (-SNRdB/10) * np.sqrt(UserPower[0]) * Nchip
rSig = ssSig + awgn(Pn, Lss, 1)

In [89]:
# 受信機
corOut = correlator(spCode, rSig);
for userCo in range(0, Nuser):
    corOut_rshp = corOut[:, userCo].reshape(Nchip, Ndata, order='F').copy()
    rData = bpskDem(corOut_rshp[0,:])
    BER = ber(data[:, userCo], rData.T)
    print('*端末{} のBER：'.format(userCo))
    print(BER)

*端末0 のBER：
0.5
*端末1 のBER：
0.0
